In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

## Setting

In [3]:
import ell
ell.init(store='./log_langgraph_', autocommit=True)

## Tools

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun


In [5]:
import math
from pydantic import Field

In [6]:
from typing import List


In [7]:
@ell.tool()
def search(query: str = Field(description="Search query to look up information.")) -> str:
    """Searches the internet for the given query and returns relevant information."""
    search_tool = DuckDuckGoSearchRun()
    return search_tool.run(query)

In [8]:
@ell.tool()
def python_repl(code: str = Field(description="Python code to execute in the REPL.")) -> str:
    """Execute Python code and return the output. Can import and use packages that are installed in the environment."""
    try:
        # Create a string buffer to capture output
        import sys
        from io import StringIO
        old_stdout = sys.stdout
        redirected_output = StringIO()
        sys.stdout = redirected_output

        # Execute the code
        exec_globals = {}
        exec(code, exec_globals)
        
        # Get the output
        sys.stdout = old_stdout
        output = redirected_output.getvalue()
        
        # If there's no stdout output, try to get the last expression value
        if not output.strip():
            last_line = code.strip().split('\n')[-1]
            try:
                result = eval(last_line, exec_globals)
                output = str(result)
            except:
                pass
        
        return output if output.strip() else "Code executed successfully with no output."
    
    except Exception as e:
        return f"Error: {str(e)}"